double check against <https://github.com/leelabcnbc/tang_jcompneuro/blob/master/results_ipynb/debug/gabor_debug/gabor_fitting_debug_simple_one_neuron.ipynb>

In [1]:
from tang_jcompneuro.io import load_split_dataset

In [2]:
# get trainer.
from tang_jcompneuro.model_fitting_gabor import get_trainer

In [3]:
datasets = load_split_dataset('MkA_Shape', 'all', False, 33, subtract_mean=True, last_val=True)
X = datasets[0]
print(X.min(), X.max(), X.mean())
X_mean = X.reshape(len(X), -1).mean(axis=1)
print(X_mean.shape, X_mean.min(), X_mean.max())
X = datasets[2]
print(X.min(), X.max(), X.mean())
X_mean = X.reshape(len(X), -1).mean(axis=1)
print(X_mean.shape, X_mean.min(), X_mean.max())

-0.85 0.983125 6.10622663544e-19
(7600,) -1.42108547152e-16 1.42108547152e-16
-0.84375 0.983125 2.38405786341e-19
(1900,) -1.42108547152e-16 1.06581410364e-16


In [4]:
trainer = get_trainer('simple')
results = trainer(datasets)

simple None
bad index 0/384


In [5]:
# pretty match what I got before last cell in previous notebook.
# difference might be due to CUDA version, PyTorch internal difference, etc.
results

{'attrs': {'corr_train': 0.57938111},
 'corr': 0.64788389,
 'model': functools.partial(<function save_model_to_hdf5_group at 0x2b8fd49bd510>, saved_params={'loc': array([[ 0.06150125],
        [-0.1337308 ]], dtype=float32), 'sigma': array([[ 0.20994249],
        [ 0.08868654]], dtype=float32), 'orientation': array([ 0.9331919], dtype=float32), 'frequency': array([ 1.17384231], dtype=float32), 'output_a': array([ 0.04350113], dtype=float32), 'output_b': 0.17815343, 'phase': array([ 4.66572618], dtype=float32), 'bias': array([ 0.10475681], dtype=float32)}),
 'y_test_hat': array([[ 0.17815343,  0.17815343,  0.17815343, ...,  0.36278272,
          0.44004801,  0.17815343]], dtype=float32)}

In [6]:
assert callable(results['model'])

In [7]:
# see if we can save models.
import h5py
from tempfile import NamedTemporaryFile
from tang_jcompneuro.gabor import load_model_from_hdf5_group

with NamedTemporaryFile() as f:
    with h5py.File(f.name) as f_tmp:
        results['model'](f_tmp)
        # probably I should not go out of this second with, as that may close f.
        # to avoid all these issues, best way would be create a temp dir, rather than a temp file.
        # but whatever.
        best_params_complex_OUT = load_model_from_hdf5_group(f_tmp)
print(best_params_complex_OUT)

{'bias': array([ 0.10475681], dtype=float32), 'frequency': array([ 1.17384231], dtype=float32), 'loc': array([[ 0.06150125],
       [-0.1337308 ]], dtype=float32), 'orientation': array([ 0.9331919], dtype=float32), 'output_a': array([ 0.04350113], dtype=float32), 'output_b': 0.17815343, 'phase': array([ 4.66572618], dtype=float32), 'sigma': array([[ 0.20994249],
       [ 0.08868654]], dtype=float32)}
